_Lambda School Data Science_

# Join datasets

Objectives
- concatenate data with pandas
- merge data with pandas

Links
- [Pandas Cheat Sheet](https://github.com/pandas-dev/pandas/blob/master/doc/cheatsheet/Pandas_Cheat_Sheet.pdf)
  - Combine Data Sets: Standard Joins
- Python Data Science Handbook
  - [Chapter 3.6](https://jakevdp.github.io/PythonDataScienceHandbook/03.06-concat-and-append.html), Combining Datasets: Concat and Append
  - [Chapter 3.7](https://jakevdp.github.io/PythonDataScienceHandbook/03.07-merge-and-join.html), Combining Datasets: Merge and Join

## Download data

We’ll work with a dataset of [3 Million Instacart Orders, Open Sourced](https://tech.instacart.com/3-million-instacart-orders-open-sourced-d40d29ead6f2)!

In [2]:
!wget https://s3.amazonaws.com/instacart-datasets/instacart_online_grocery_shopping_2017_05_01.tar.gz

--2019-03-26 18:28:59--  https://s3.amazonaws.com/instacart-datasets/instacart_online_grocery_shopping_2017_05_01.tar.gz
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.216.99.109
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.216.99.109|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 205548478 (196M) [application/x-gzip]
Saving to: ‘instacart_online_grocery_shopping_2017_05_01.tar.gz.2’

instacart_online_gr 100%[===================>] 196.03M  74.1MB/s    in 2.6s    

2019-03-26 18:29:02 (74.1 MB/s) - ‘instacart_online_grocery_shopping_2017_05_01.tar.gz.2’ saved [205548478/205548478]



In [3]:
!tar --gunzip --extract --verbose --file=instacart_online_grocery_shopping_2017_05_01.tar.gz

instacart_2017_05_01/
instacart_2017_05_01/._aisles.csv
instacart_2017_05_01/aisles.csv
instacart_2017_05_01/._departments.csv
instacart_2017_05_01/departments.csv
instacart_2017_05_01/._order_products__prior.csv
instacart_2017_05_01/order_products__prior.csv
instacart_2017_05_01/._order_products__train.csv
instacart_2017_05_01/order_products__train.csv
instacart_2017_05_01/._orders.csv
instacart_2017_05_01/orders.csv
instacart_2017_05_01/._products.csv
instacart_2017_05_01/products.csv


In [4]:
%cd instacart_2017_05_01

/content/instacart_2017_05_01


## Goal: Reproduce this example

The first two orders for user id 1:

In [5]:
from IPython.display import display, Image
url = 'https://cdn-images-1.medium.com/max/1600/1*vYGFQCafJtGBBX5mbl0xyw.png'
example = Image(url=url, width=600)

display(example)

## Load data

Here's a list of all six CSV filenames

In [6]:
!ls -lh

total 681M
-rw-r--r-- 1 502 staff 2.6K May  2  2017 aisles.csv
-rw-r--r-- 1 502 staff  270 May  2  2017 departments.csv
-rw-r--r-- 1 502 staff 551M May  2  2017 order_products__prior.csv
-rw-r--r-- 1 502 staff  24M May  2  2017 order_products__train.csv
-rw-r--r-- 1 502 staff 104M May  2  2017 orders.csv
-rw-r--r-- 1 502 staff 2.1M May  2  2017 products.csv


For each CSV
- Load it with pandas
- Look at the dataframe's shape
- Look at its head (first rows)
- `display(example)`
- Which columns does it have in common with the example we want to reproduce?

In [0]:
import pandas as pd

### aisles

In [0]:
aisles = pd.read_csv('aisles.csv')

Aisles doesnt have any columns we need

### departments

In [0]:
departments = pd.read_csv('departments.csv')

Departments doesnt have any columns we need

### order_products__prior

In [0]:
order_products__prior = pd.read_csv('order_products__prior.csv')

In [0]:
'''
order_products_prior columns we need:

-order_id
-product_id
-add_to_cart_order

'''

### order_products__train

In [0]:
order_products__train = pd.read_csv('order_products__train.csv')

In [0]:
'''
order_products__train columns we need:

-order_id
-product_id
add_to_cart_order

'''

### orders

In [0]:
orders = pd.read_csv('orders.csv')

In [0]:
'''
orders columns we need:

-order_id 
-user_id 
-order_number 
-order_dow 
-order_hour_of_day

'''

### products

In [0]:
products = pd.read_csv('products.csv')

In [0]:
'''
products columns we need:

-product_id
-product_name

'''

## Concatenate order_products__prior and order_products__train

In [0]:
order_products = pd.concat([order_products__prior, order_products__train])


## Get a subset of orders — the first two orders for user id 1

In [0]:
columns = ['user_id', 'order_id', 'order_number', 'order_dow', 'order_hour_of_day']

user1 = orders[orders['user_id'] == 1]

first2 = user1[columns].head(2)


In [36]:
first2

,user_id,order_id,order_number,order_dow,order_hour_of_day
0,1,2539329,1,2,8
1,1,2398795,2,3,7


## Merge dataframes

In [16]:
orders.head()

orders = orders[columns]
orders.head()

,user_id,order_id,order_number,order_dow,order_hour_of_day
0,1,2539329,1,2,8
1,1,2398795,2,3,7
2,1,473747,3,3,12
3,1,2254736,4,4,7
4,1,431534,5,4,15


In [0]:
order_products = order_products.drop(columns='reordered')

In [0]:
merged = pd.merge(first2, order_products)

In [19]:
merged.head(20)

,user_id,order_id,order_number,order_dow,order_hour_of_day,product_id,add_to_cart_order
0,1,2539329,1,2,8,196,1
1,1,2539329,1,2,8,14084,2
2,1,2539329,1,2,8,12427,3
3,1,2539329,1,2,8,26088,4
4,1,2539329,1,2,8,26405,5
5,1,2398795,2,3,7,196,1
6,1,2398795,2,3,7,10258,2
7,1,2398795,2,3,7,12427,3
8,1,2398795,2,3,7,13176,4
9,1,2398795,2,3,7,26088,5


In [20]:
#merge with columns from products

final = pd.merge(merged, products[['product_id', 'product_name']])
final = final.sort_values(by=['order_number', 'add_to_cart_order']).set_index('user_id')
final.head(20)

,order_id,order_number,order_dow,order_hour_of_day,product_id,add_to_cart_order,product_name
user_id,,,,,,,
1,2539329,1,2,8,196,1,Soda
1,2539329,1,2,8,14084,2,Organic Unsweetened Vanilla Almond Milk
1,2539329,1,2,8,12427,3,Original Beef Jerky
1,2539329,1,2,8,26088,4,Aged White Cheddar Popcorn
1,2539329,1,2,8,26405,5,XL Pick-A-Size Paper Towel Rolls
1,2398795,2,3,7,196,1,Soda
1,2398795,2,3,7,10258,2,Pistachios
1,2398795,2,3,7,12427,3,Original Beef Jerky
1,2398795,2,3,7,13176,4,Bag of Organic Bananas


In [0]:
display(example)

# Assignment

These are the top 10 most frequently ordered products. How many times was each ordered? 

1. Banana
2. Bag of Organic Bananas
3. Organic Strawberries
4. Organic Baby Spinach 
5. Organic Hass Avocado
6. Organic Avocado
7. Large Lemon 
8. Strawberries
9. Limes 
10. Organic Whole Milk

First, write down which columns you need and which dataframes have them.

Next, merge these into a single dataframe.

Then, use pandas functions from the previous lesson to get the counts of the top 10 most frequently ordered products.

## Stretch challenge

The [Instacart blog post](https://tech.instacart.com/3-million-instacart-orders-open-sourced-d40d29ead6f2) has a visualization of "**Popular products** purchased earliest in the day (green) and latest in the day (red)." 

The post says,

> "We can also see the time of day that users purchase specific products.

> Healthier snacks and staples tend to be purchased earlier in the day, whereas ice cream (especially Half Baked and The Tonight Dough) are far more popular when customers are ordering in the evening.

> **In fact, of the top 25 latest ordered products, the first 24 are ice cream! The last one, of course, is a frozen pizza.**"

Your challenge is to reproduce the list of the top 25 latest ordered popular products.

We'll define "popular products" as products with more than 2,900 orders.

In [0]:
#set columns we need from each df:

orders_cols = ['user_id', 'order_id', 'order_number', 'order_hour_of_day']
orders_products_cols = ['order_id', 'product_id']
products_cols = ['product_id', 'product_name']



In [0]:
#merge orders and order_products (one to many):

order_and_order_products = pd.merge(orders[orders_cols], order_products[orders_products_cols])


#merge w products

combined = pd.merge(order_and_order_products, products[products_cols])



In [46]:
product_order_counts = combined.product_name.value_counts()

product_order_counts.head(10)

Banana                    491291
Bag of Organic Bananas    394930
Organic Strawberries      275577
Organic Baby Spinach      251705
Organic Hass Avocado      220877
Organic Avocado           184224
Large Lemon               160792
Strawberries              149445
Limes                     146660
Organic Whole Milk        142813
Name: product_name, dtype: int64

# Stretch Goal

In [48]:
popular = product_order_counts[product_order_counts >= 2900]
popular.shape

(2053,)

In [63]:
combined_popular = combined[combined['product_name'].isin(popular.index)]
combined_popular.head()

,user_id,order_id,order_number,order_hour_of_day,product_id,product_name
0,1,2539329,1,8,196,Soda
1,1,2398795,2,7,196,Soda
2,1,473747,3,12,196,Soda
3,1,2254736,4,7,196,Soda
4,1,431534,5,15,196,Soda


In [0]:
'''Group by product name, grab the order hour of day column and take the mean 
for each product, sort by latest in the day, and grab the top 25 w head and 
index then format as list'''

top25_latest = list(combined_popular.groupby('product_name')
                    .order_hour_of_day.mean()
                    .sort_values(ascending=False)
                    .head(25)
                    .index)

In [62]:
print(top25_latest)

['Half Baked Frozen Yogurt', 'The Tonight Dough™ Ice Cream', 'Americone Dream® Ice Cream', 'Half Baked® Ice Cream', 'Phish Food® Ice Cream', 'Milk and Cookies Ice Cream', 'Sicilian Pistachio', 'Chocolate Peanut Butter Cup Gelato', 'Birthday Cake Light Ice Cream', 'Cherry Garcia Ice Cream', 'Strawberry Ice Cream', 'Vanilla Bean Light Ice Cream', 'Coffee Ice Cream', 'Roman Raspberry Sorbetto', 'Mediterranean Mint Gelato', 'Caramel Cookie Crunch Gelato', 'Chocolate Chip Cookie Dough Ice Cream', 'Mint Chip', 'Coconut Almond Minis Frozen Dessert Bars', 'Mint Chocolate Chip Ice Cream', 'Chocolate Peanut Butter Ice Cream', 'Vanilla Milk Chocolate Almond Ice Cream Bars', 'Sea Salt Caramel Gelato', 'Vanilla Bean Ice Cream', 'Rising Crust Pepperoni Pizza']
